There are **four key ideas** behind Lasagna AI:

1. We want to build **layered** agents!
2. We want it to be **plugable** (both _models_ and _agents_ plug together in all directions).
3. We want to deploy stuff into **production**!
4. We want **type safety**!

## Prerequisite Knowledge

### Python `asyncio`

The production-mindedness of Lasagna AI means it's fully async, so it plays nicely with remote APIs and modern Python web frameworks. If `asyncio` is new to you, read [Python & Async Simplified](https://www.aeracode.org/2018/02/19/python-async-simplified/).

### Functional Programming

The pipeline-nature of AI systems lends itself toward **functional programming**. If functional programming is new to you, watch [Dear Functional Bros](https://youtu.be/nuML9SmdbJ4?si=eQ6Qla11k3ayJD79) and read [Functional Programming](misc/functional_programming.ipynb).

A quick recap of functional programming:

- State is immutable:
    - Want to modify something? **TOO BAD!**
    - Instead, make a copy (_with your modifications applied_).
- Pass lots of functions as parameters to other functions:
    - We think it's **fun** and **cool**.
    - You will too once you get used to the idea.

### Basic idea of Lasagna's Layered Agents

With Lasagna AI you'll build several simple agents, then compose them together into a layered multi-agent system! Yay! 🥳

You can skip for now, but _eventually_ you'll want to read:

- [The Lasagna `Agent`](what_is_an_agent/agent.ipynb)
- [The `AgentRun` type](what_is_an_agent/type_agentrun.ipynb)

## Hello Lasagna

We're so close to writing code! Hang tight. 😎

### It's all about the `Agent`

The **Lasagna Agent** is just a _callable_ that takes three parameters:

- `model`: The _model_ that is available for the agent's use (most commonly this will be a _Large Language Model_; aka an LLM).
- `event_callback`: This is a callback for streaming!
    - It's generic, so you can invent new event types if you need.
    - It also emits events at agent-boundaries, which can be useful.
- `prev_runs`: In a multi-turn chat system, this will be a list of "previous runs" of this agent; that is, this is your conversation history!

Here is your first agent:

In [ ]:
from lasagna import Model, EventCallback, AgentRun

async def my_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    raise RuntimeError("not implemented")

You can make it a _callable object_ (rather than a _function_), if you'd want, like this:

In [ ]:
class MyAgent:
    def __init__(self):
        pass

    async def __call__(
        model: Model,
        event_callback: EventCallback,
        prev_runs: list[AgentRun],
    ) -> AgentRun:
        raise RuntimeError("not implemented")

my_agent = MyAgent()

### The `Agent`'s job

The most _basic_ agent will do this:

1. Look through the conversation history (supplied in the `prev_runs` parameter) and extract all the messages from that history.
2. Invoke `model` with those messages, and grab the _new_ message(s) that the model generates.
3. Wrap those _new_ message(s) up into an `AgentRun`, and return it.

(We'll discuss more _complex_ agent behaviors later.)

So, the most _basic_ agent looks like this:

In [ ]:
from lasagna import recursive_extract_messages, flat_messages

async def my_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    messages = recursive_extract_messages(prev_runs, from_layered_agents=False)
    new_messages = await model.run(event_callback, messages, tools=[])
    this_run = flat_messages('my_agent', new_messages)
    return this_run

### "Binding" the `Agent`

### Test in the Terminal

## Put it all together!

Here is everything above in one cell, so you can easily run it yourself:

In [ ]:
from lasagna import (
    known_models,
    Model,
    EventCallback,
    AgentRun,
    recursive_extract_messages,
    flat_messages,
)

from lasagna.tui import tui_input_loop

from dotenv import load_dotenv; load_dotenv()


@known_models.BIND_OPENAI_gpt_4o_mini()
async def my_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    messages = recursive_extract_messages(prev_runs, from_layered_agents=False)
    new_messages = await model.run(event_callback, messages, tools=[])
    this_run = flat_messages('my_agent', new_messages)
    return this_run


SYSTEM_PROMPT = "You are grumpy."


await tui_input_loop(my_agent, SYSTEM_PROMPT)  # type: ignore[top-level-await]